In [1]:
import numpy as np 
import warnings
import import_ipynb
import Pipeline_Control as pp2
import DeepBind_Control as db22
import os
import pandas as pd
import sys
import matplotlib.pyplot as pl
import subprocess
import gc
import random
from Bio import SeqIO
warnings.filterwarnings("ignore")
from guppy import hpy; 


dir(db22)
dir(pp2)

importing Jupyter notebook from Pipeline_Control.ipynb
importing Jupyter notebook from DeepBind_Control.ipynb


/home/abhinav/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['__builtins__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 'get_ipython',
 'pipe']

In [2]:
def remove_1_8_21_from_class1_file(class1file):
    
    file1, file_extension1 = os.path.splitext(class1file)
    
    f2 = "inter_bed/class0_11files_only1821_test.bed"
    
    print("here1: ",file1)
    
    #initializing filenames
    class1_test_bed= "inter_bed/class1_"+file1+"_test.bed"

    #copy entries of chr1 to class1_test_bed
    command = '''awk '/chr1\t/' '''+class1file+''' >> '''+class1_test_bed
    subprocess.call(command, shell=True)
    
    #copy entries of chr8 to class1_test_bed
    command = '''awk '/chr8\t/' '''+class1file+''' >> '''+class1_test_bed
    subprocess.call(command, shell=True)
    
    #copy entries of chr21 to class1_test_bed
    command = '''awk '/chr21\t/' '''+class1file+''' >> '''+class1_test_bed
    subprocess.call(command, shell=True)
    
    
    #removing intersections between overall class0_test (chr1,8,21 of 11 files) and class1_test_bed (chr1,8,21 of one file like IFF)
    count, num_in_class0, num_in_class1 = pp2.pipe(class1_test_bed,f2)
    
    
    #remove 1, 8, 21 from class1 train and return 
    command='''sed -i '/chr1\t/d' ./'''+class1file
    subprocess.call(command, shell=True)

    command='''sed -i '/chr8/d' ./'''+class1file
    subprocess.call(command, shell=True)

    command='''sed -i '/chr21/d' ./'''+class1file
    subprocess.call(command, shell=True)
    
    return class1file,  count, num_in_class0, num_in_class1

In [3]:
def converter(sequence):
    new_sequence=[]
    for i in range(len(sequence)):
        if sequence[i]=='a' or sequence[i]=='A':
            new_sequence.append('T')
        elif sequence[i]=='c' or sequence[i]=='C':
            new_sequence.append('G')
        elif sequence[i]=='g' or sequence[i]=='G':
            new_sequence.append('C')
        elif sequence[i]=='t' or sequence[i]=='T':
            new_sequence.append('A')
        
            
    str1 = ''.join(new_sequence)
    str1=''.join(reversed(str1))
    
    return str1

In [4]:
#filenames1 = ['ENCFF183UQD_H3K4me1.bed','ENCFF322IFF_H3K27me3.bed','ENCFF498CMP_H3K36me3.bed']
#filenames1 = ['ENCFF127XXD_H3K4me3.bed','ENCFF099LMD_H3K4me2.bed', 'ENCFF139CKE_H4K20me1.bed', 'ENCFF183UQD_H3K4me1.bed','ENCFF322IFF_H3K27me3.bed','ENCFF498CMP_H3K36me3.bed', 'ENCFF578UBP_H3K27ac.bed', 'ENCFF624XRN_H2AFZ.bed', 'ENCFF737AMS_H3K4me3.bed', 'ENCFF894VEM_H3K9me3.bed']
#filenames1 = ['ENCFF578UBP_H3K27ac.bed', 'ENCFF624XRN_H2AFZ.bed', 'ENCFF737AMS_H3K4me3.bed', 'ENCFF894VEM_H3K9me3.bed','ENCFF127XXD_H3K4me3.bed','ENCFF099LMD_H3K4me2.bed', 'ENCFF139CKE_H4K20me1.bed','ENCFF183UQD_H3K4me1.bed','ENCFF322IFF_H3K27me3.bed','ENCFF498CMP_H3K36me3.bed']

filenames1=['ENCFF322IFF_H3K27me3.bed']


In [5]:
df_final = pd.DataFrame(columns=['File1','File2','Accuracy', 'Scores_Array'])

print len(filenames1)


1


In [6]:
def classify(f1):
    count=0
    f1,  num_intersections, num_in_class0, num_in_class1 = remove_1_8_21_from_class1_file(f1)     #f1=class1_train.bed w/o 1,8,21
    f2 = "./class0.bed"        


    print("f1: ",f1)
    print("f2: ",f2)

    num_intersections_train, num_in_class0_train, num_in_class1_train = pp2.pipe("./"+f1,f2)

    file1, file_extension1 = os.path.splitext(f1)
    f1=file1.split("/")
    print("f1 is:",f1[0])

    class0_train= "inter_fa/class0.fa"
    class1_train= "inter_fa/"+f1[0]+".fa"
    class0_test="inter_fa/class0_11files_only1821_test.fa"
    class1_test = "inter_fa/class1_"+f1[0] +"_test.fa" 

    list0=[]
    list1=[]

    #making dataframe for train data
    for record in SeqIO.parse(class0_train, "fasta"):
        if 'N' not in str(record.seq):
            sequence=str(record.seq)+"NNNNNNNNNN"+str(converter(record.seq))
            list0.append(sequence)
    for record in SeqIO.parse(class1_train, "fasta"):
        if 'N' not in str(record.seq):
            sequence=str(record.seq)+"NNNNNNNNNN"+str(converter(record.seq))
            list1.append(sequence)
    target=[]

#     random.shuffle(list0)
#     random.shuffle(list1)
#     list0= list0[:20000]
#     list1= list1[:20000]
    
    for t in range(len(list0)):
        target.append(0)

    for m in range(len(list1)):
        target.append(1)

    sequences=list0+list1
    
    

    dictionary = {'seq': sequences, 'target': target}
    df_train = pd.DataFrame(dictionary)

    
    del list0
    del list1
    list0=[]
    list1=[]

    #making dataframe for test data
    for record in SeqIO.parse(class0_test, "fasta"):
        if 'N' not in str(record.seq):
            sequence=str(record.seq)+"NNNNNNNNNN"+str(converter(record.seq))
            list0.append(sequence)
    for record in SeqIO.parse(class1_test, "fasta"):
        if 'N' not in str(record.seq):
            sequence=str(record.seq)+"NNNNNNNNNN"+str(converter(record.seq))
            list1.append(sequence)
    target=[]

#     random.shuffle(list0)
#     random.shuffle(list1)
#     list0= list0[:10000]
#     list1= list1[:10000]

    for t in range(len(list0)):
        target.append(0)

    for m in range(len(list1)):
        target.append(1)

    sequences=list0+list1

    dictionary = {'seq': sequences, 'target': target}
    df_test = pd.DataFrame(dictionary)

    print(df_train.shape)
    print(df_test.shape)


    #calling deepbind
    accuracy,scores= db22.deepbind1(df_train,df_train,count)
    #         print("Num intersections in 30%: ",num_intersections_in30,"\n")
    #         print("Num in class0: ",num_in_class0,"\n")
    #         print("Num in class1: ",num_in_class1,"\n")
    print("File1: ",f1,"\n")
    print("File2: ",f2,"\n")
    print("Accuracy: ",accuracy,"\n")
    print("Prob: ",scores,"\n")
    print ("Scores size array" , len(scores), "\n")
    #print("Probability: ",scores,"\n")
    df_final.loc[count,['File1']] = str(f1)
    df_final.loc[count, ['File2']] = str(f2)
    df_final.loc[count, ['Accuracy']] = accuracy
    df_final.loc[count, ['Scores_Array']] = str(scores)



    file2, file_extension2 = os.path.splitext(f2)
    f2=file2.split("/")
    print("f2 is:",f2[1])


    #histogram stuff
    title1="Class0 num: "+str(num_in_class0)+" Class1 num: "+str(num_in_class1)+" Num. of intersections: "+str(num_intersections)
    plot_title= "histograms/"+str(f1)+"_"+str(f2[1])+".png"
    pl.figure(figsize=(7,7))

    n1,bins1,patches1 = pl.hist(scores[0:num_in_class0],bins=10, density = False, edgecolor='black',alpha=0.5,label = "class0-red")
    n2,bins2,patches2 = pl.hist(scores[num_in_class0+1:],bins=10, density = False, edgecolor='black',alpha=0.5, label = "class1-black")
    for p in patches1:
        pl.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()),fontsize=12, color='red', ha='center', va='bottom')
    for p in patches2:
        pl.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()),fontsize=12, color='black', ha='center', va='bottom')
    pl.title(title1)
    pl.xlabel("Predicted probabilities")
    pl.ylabel("Frequency")
    pl.legend(loc='upper left')
    pl.savefig(plot_title)
    pl.clf()
    print("Printing Stats")
    h=hpy()
    print(h.heap())
    print("\n")
    del list0
    del list1
    del df_train
    del df_test
    gc.collect()
    

    count=count+1
    
    return accuracy, scores

In [9]:
accuracy,scores = classify('ENCFF578UBP_H3K27ac.bed')

('here1: ', 'ENCFF578UBP_H3K27ac')
('in pipeline: f1', 'inter_bed/class1_ENCFF578UBP_H3K27ac_test.bed')
('in pipeline: f2', 'inter_bed/class0_11files_only1821_test.bed')
('Filename 1 in pipe:', ['inter_bed', 'class1_ENCFF578UBP_H3K27ac_test'])
('Filename 2 in pipe:', ['inter_bed', 'class0_11files_only1821_test'])
0
('f1: ', 'ENCFF578UBP_H3K27ac.bed')
('f2: ', './class0.bed')
('in pipeline: f1', './ENCFF578UBP_H3K27ac.bed')
('in pipeline: f2', './class0.bed')
('Filename 1 in pipe:', ['.', 'ENCFF578UBP_H3K27ac'])
('Filename 2 in pipe:', ['.', 'class0'])
0
('f1 is:', 'ENCFF578UBP_H3K27ac')
(145651, 2)
(28448, 2)
('Shape is: ', (145651, 2))
('Shape test :', (145651, 2))
[[[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]
  ...
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 0. 1. 0.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  ...
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 ...

 [[0. 0

<Figure size 504x504 with 0 Axes>

In [10]:
print(accuracy)

0.6960098914231158


In [11]:
print(scores)

[[0.15681487]
 [0.34334344]
 [0.6244261 ]
 ...
 [0.66407454]
 [0.3862894 ]
 [0.4947845 ]]
